In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [2]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    # print(data)
    return data[:,:-1], data[:,-1]

In [3]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
X_test[0], y_test[0]

(array([5.5, 2.3, 4. , 1.3]), 1.0)

In [5]:
# Gaussian Naive Bayes Model

class NaiveBayes:
    
    def __init__(self):
        self.model = None
    
    # expectation
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))
    
    # standard deviation
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow((x - avg), 2) for x in X])) / float(len(X))
    
    # pdf
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    
    # handle X_train
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summaries
    
    def train(self, X, y):
        labels = list(set(y))
        data = {label: [] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {
            label: self.summarize(value)
            for label, value in data.items()
        }
        print('GaussianNB train done!')
        
    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev)
#                 print(probabilities)
        return probabilities
    
    def predict(self, X_test):
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label
    
    def test_acc(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1
        return right / float(len(X_test))

In [6]:
model = NaiveBayes()

In [7]:
model.train(X_train, y_train)

GaussianNB train done!


In [8]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))

0.0


In [9]:
model.test_acc(X_test, y_test)

1.0